# PART2 : 以 jieba 探索文本主題 (歌詞文本)

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import jieba.analyse
import jieba
import codecs

## 資料前處理

In [2]:
# 斷詞存成文件
jieba.set_dictionary("jieba_dict/dict.txt.big")

wf = codecs.open("lyrics/lyrics_cut_mayday.dataset", "w","utf-8")
with open("lyrics/mayday.txt", "r") as f:
    for line in f:
        words = jieba.cut(line)
        wf.write(" ".join(words))
        #print(" ".join(words))
wf.close()    

Building prefix dict from /Users/youngmihuang/Desktop/PyLadies/jieba_dict/dict.txt.big ...
Loading model from cache /var/folders/c6/vq7n4xz94bqdm56d8hb739kr0000gn/T/jieba.u61de2cbc86a45c8f420ba10366a9081d.cache
Loading model cost 2.415 seconds.
Prefix dict has been built succesfully.


In [ ]:
# 查看：斷詞前處理
with open("lyrics/lyrics_cut_mayday.dataset") as fn:
    for line in fn:
        print(line)

In [3]:
# 載入同義字
word_net = []
with open("lyrics/word_net.txt", "r") as f1:
    for line in f1:
        word_net.append(line)

word_net = sorted(set(word_net))
word_net_dic = {}

for word in word_net:
    word_s = word.split()
    word_net_dic[word_s[0]] = word_s[1]


### 將斷好詞的檔案讀進來，如果字詞有出現在 word_net當中，則替換成同義字；若無，則保留原本字義

In [4]:
# 將資料處理後的檔案存檔
wf = open("lyrics/lyrics_word_net_mayday.dataset", "w")

with open("lyrics/lyrics_cut_mayday.dataset", "r") as f2:
    for line in f2:
        line_words = line.split()
        line_lyrics = ""
        for line_word in line_words:
            if line_word in word_net_dic:
                line_lyrics = line_lyrics + word_net_dic[line_word] + ' '
            else:
                line_lyrics = line_lyrics + line_word + ' '
        #print(line_lyrics+"\n")
        wf.write(line_lyrics+"\n")

wf.close()

# 主題建模開始
## 1. 語料庫與向量空間 ( Corpora and Vector Spaces )

In [5]:
# see logging events
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
import os
from gensim import corpora, models, similarities
#from six import iteritems

2017-11-04 22:03:40,018 : INFO : 'pattern' package not found; tag filters are not available for English


In [6]:
# 移除常見字
with open("lyrics/stop_words.txt") as f:
    stop_word_content = f.readlines()
stop_word_content = [x.strip() for x in stop_word_content] #strip: 移除頭尾空格、中間不會
stop_word_content = " ".join(stop_word_content)

# 建立本次文檔的語料庫(字典)
# 將文檔裡的詞袋給予編號
dictionary = corpora.Dictionary(document.split() for document in open("lyrics/lyrics_word_net_mayday.dataset"))
stoplist = set(stop_word_content.split())
stop_ids = [dictionary.token2id[stopword] for stopword in stoplist
            if stopword in dictionary.token2id] #dictionary.token2id: 代表什麼字詞對應到什麼id，有幾個id就代表有幾維向量空間
dictionary.filter_tokens(stop_ids) # 移除停用字
dictionary.compactify() #remove faps in id sequence after worfs that were removed
dictionary.save("lyrics/lyrics_mayday.dict")



2017-11-04 22:03:46,989 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2017-11-04 22:03:47,006 : INFO : built Dictionary(2221 unique tokens: ['會', '不會', '有', '一天', '時間']...) from 36 documents (total 8333 corpus positions)
2017-11-04 22:03:47,015 : INFO : saving Dictionary object under lyrics/lyrics_mayday.dict, separately None
2017-11-04 22:03:47,019 : INFO : saved lyrics/lyrics_mayday.dict


In [ ]:
# 查看：序列化的結果
for word,index in dictionary.token2id.items(): 
    print(word +" id:"+ str(index))

In [7]:
texts = [[word for word in document.split() if word not in stoplist]
         for document in open("lyrics/lyrics_word_net_mayday.dataset")]

# 移除只出現一次的字詞
# from collections import defaultdict
# frequency = defaultdict(int)
# for text in texts:
#     for token in text:
#         frequency[token] += 1

# texts = [[token for token in text if frequency[token] > 1]
#          for text in texts]

# 將 corpus 序列化
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize("lyrics/lyrics_mayday.mm", corpus) # Corpus in Matrix Market format 
# 其他序列化方法：Other formats include Joachim’s SVMlight format, Blei’s LDA-C format and GibbsLDA++ format.

2017-11-04 22:03:53,327 : INFO : storing corpus in Matrix Market format to lyrics/lyrics_mayday.mm
2017-11-04 22:03:53,329 : INFO : saving sparse matrix to lyrics/lyrics_mayday.mm
2017-11-04 22:03:53,330 : INFO : PROGRESS: saving document #0
2017-11-04 22:03:53,352 : INFO : saved 36x2213 matrix, density=5.141% (4096/79668)
2017-11-04 22:03:53,354 : INFO : saving MmCorpus index to lyrics/lyrics_mayday.mm.index


## 2. tf — idf 轉換 與 創建 LSI 模型 （Topics and Transformations）
- ### tfidf 
- ### 透過 tfidf 創建 LSI 模型

In [8]:
# 載入語料庫
if (os.path.exists("lyrics/lyrics_mayday.dict")):
    dictionary = corpora.Dictionary.load("lyrics/lyrics_mayday.dict")
    corpus = corpora.MmCorpus("lyrics/lyrics_mayday.mm") # 將數據流的語料變為內容流的語料
    print("Used files generated from first tutorial")
else:
    print("Please run first tutorial to generate data set")

2017-11-04 22:03:56,544 : INFO : loading Dictionary object from lyrics/lyrics_mayday.dict
2017-11-04 22:03:56,548 : INFO : loaded lyrics/lyrics_mayday.dict
2017-11-04 22:03:56,549 : INFO : loaded corpus index from lyrics/lyrics_mayday.mm.index
2017-11-04 22:03:56,550 : INFO : initializing corpus reader from lyrics/lyrics_mayday.mm
2017-11-04 22:03:56,552 : INFO : accepted corpus with 36 documents, 2213 features, 4096 non-zero entries


Used files generated from first tutorial


In [9]:
# 創建 tfidf model
tfidf = models.TfidfModel(corpus)
# 轉為向量表示
corpus_tfidf = tfidf[corpus]

2017-11-04 22:03:59,683 : INFO : collecting document frequencies
2017-11-04 22:03:59,686 : INFO : PROGRESS: processing document #0
2017-11-04 22:03:59,704 : INFO : calculating IDF weights for 36 documents and 2212 features (4096 matrix non-zeros)


### 建立lsi模型的時候：需要給定tfidf生成的語料庫(corpus: lyrics.mm)、給定字典(dictionary: lyrics.dict)、和制定主題數

In [10]:
# 創建 LSI model
lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=10)
corpus_lsi = lsi[corpus_tfidf] # LSI潛在語義索引
lsi.save('lyrics/lyrics_mayday.lsi')
corpora.MmCorpus.serialize('lyrics/lsi_corpus_mayday.mm', corpus_lsi)
print("LSI topics:")
lsi.print_topics(5)

2017-11-04 22:04:03,299 : INFO : using serial LSI version on this node
2017-11-04 22:04:03,301 : INFO : updating model with new documents
2017-11-04 22:04:03,333 : INFO : preparing a new chunk of documents
2017-11-04 22:04:03,338 : INFO : using 100 extra samples and 2 power iterations
2017-11-04 22:04:03,339 : INFO : 1st phase: constructing (2213, 110) action matrix
2017-11-04 22:04:03,341 : INFO : orthonormalizing (2213, 110) action matrix
2017-11-04 22:04:03,482 : INFO : 2nd phase: running dense svd on (110, 36) matrix
2017-11-04 22:04:03,485 : INFO : computing the final decomposition
2017-11-04 22:04:03,486 : INFO : keeping 10 factors (discarding 65.952% of energy spectrum)
2017-11-04 22:04:03,490 : INFO : processed documents up to #36
2017-11-04 22:04:03,492 : INFO : topic #0(1.432): 0.204*"噢" + 0.132*"一次" + 0.127*"我倆" + 0.113*"一天" + 0.106*"兄弟" + 0.105*"著" + 0.105*"最" + 0.098*"這" + 0.096*"和" + 0.094*"在"
2017-11-04 22:04:03,493 : INFO : topic #1(1.163): 0.725*"噢" + 0.282*"兄弟" + 0.13

LSI topics:


[(0,
  '0.204*"噢" + 0.132*"一次" + 0.127*"我倆" + 0.113*"一天" + 0.106*"兄弟" + 0.105*"著" + 0.105*"最" + 0.098*"這" + 0.096*"和" + 0.094*"在"'),
 (1,
  '0.725*"噢" + 0.282*"兄弟" + 0.130*"oh" + 0.123*"一次" + 0.103*"唔" + -0.089*"最" + -0.088*"突然" + 0.081*"最好" + 0.079*"一個" + 0.074*"這樣"'),
 (2,
  '-0.194*"決定" + -0.187*"動次" + -0.186*"oh" + -0.171*"love" + -0.133*"ing" + -0.128*"真正" + 0.125*"我倆" + -0.119*"無望" + 0.112*"任意" + 0.109*"無限"'),
 (3,
  '0.367*"oh" + 0.242*"動次" + -0.213*"love" + -0.177*"無望" + -0.146*"ing" + 0.141*"突然" + -0.128*"路" + -0.118*"I" + -0.118*"這款" + -0.118*"法度"'),
 (4,
  '0.256*"oh" + -0.200*"一次" + 0.199*"love" + -0.198*"倔強" + 0.192*"動次" + 0.172*"ing" + -0.163*"啦" + 0.111*"無望" + 0.104*"噢" + -0.102*"自己"')]

In [ ]:
# 查看：每一首歌在各主題的佔比計算
for doc in corpus_lsi:
    print(doc)

## 3. 相似度計算 （Similarity interface）
- ###  輸入歌曲
- ###  建立索引
- ###  相似歌曲輸出

In [11]:
# 基於tfidf-> lsi 的文本相似度分析
doc = "想 把 你 寫成 一首歌 想養 一隻 貓 想要 回到 每個 場景 撥慢 每 隻 錶 我們 在 小孩 和 大人 的 轉角 蓋 一座 城堡 我們 好好 好 到 瘋 掉 像 找回 失散多年 雙胞 生命 再長 不過 煙火 落下 了 眼角 世界 再大 不過 你 我 凝視 的 微笑 在 所有 流逝 風景 與 人群 中 你 對 我 最好 一切 好好 是否 太好 沒有 人 知道 你 和 我 背著 空空 的 書包 逃出 名為 日常 的 監牢 忘 了 要 長大 忘 了 要 變老 忘 了 時間 有腳 最 安靜 的 時刻 回憶 總是 最 喧囂 最 喧囂 的 狂歡 寂寞 包圍 著 孤島 還以 為 馴服 想念 能 陪伴 我 像 一隻 家貓 它 就 窩 在 沙發 一角 卻 不肯 睡著 你 和 我 曾 有 滿滿的 羽毛 跳 著名 為 青春 的 舞蹈 不 知道 未來 不 知道 煩惱 不知 那些 日子 會 是 那麼 少 時間 的 電影 結局 才 知道 原來 大人 已 沒有 童謠 最後 的 叮嚀 最後 的 擁抱 我們 紅著 眼笑 我們 都 要 把 自己 照顧 好 好 到 遺憾 無法 打擾 好好 的 生活 好好 的 變老 好好 假裝 我 已經 把 你 忘掉 "
vec_bow = dictionary.doc2bow(doc.split()) # 把doc語料庫轉為一個一個詞包
vec_lsi = lsi[vec_bow] # 用前面建好的 lsi 模型去計算這一篇歌詞 (input: 斷詞後的詞包、output: 20個主題成分)
print(vec_lsi)

[(0, 7.2705663600217516), (1, -2.7284780730200788), (2, -0.8223383837366246), (3, 2.1469048274926479), (4, -0.65830306598173516), (5, -3.901771775577473), (6, -2.5031445409605637), (7, 1.8386513217392493), (8, 1.7113198568988774), (9, 2.8521822914392785)]


In [12]:
# 建立索引
index = similarities.MatrixSimilarity(lsi[corpus]) 
index.save("lyrics/lyrics_mayday.index") 

# 計算相似度（前五名）
sims = index[vec_lsi] 
sims = sorted(enumerate(sims), key=lambda item: -item[1])
print(sims[:5])

2017-11-04 22:04:16,517 : WARNING : scanning corpus to determine the number of features (consider setting `num_features` explicitly)
2017-11-04 22:04:16,541 : INFO : creating matrix with 36 documents and 10 features
2017-11-04 22:04:16,568 : INFO : saving MatrixSimilarity object under lyrics/lyrics_mayday.index, separately None
2017-11-04 22:04:16,570 : INFO : saved lyrics/lyrics_mayday.index


[(2, 0.99679214), (25, 0.87377143), (29, 0.84817392), (33, 0.67252529), (30, 0.66729981)]


### 輸出：索引結果對應前五名相似歌詞

In [13]:
lyrics = [];
fp = open("lyrics/lyrics_word_net_mayday.dataset") # 斷詞後的歌詞
#fp = open("lyrics/lyrics.dataset") # 看完整的歌詞
for i, line in enumerate(fp):
    lyrics.append(line)
fp.close()

for lyric in sims[:5]:
    print("\n相似歌詞：",  lyrics[lyric[0]])
    print("相似度：",  lyric[1])


相似歌詞： 想 把 你 寫成 一首歌 想養 一隻 貓 想要 回到 每個 場景 撥慢 每 隻 錶 我倆 在 小孩 和 大人 的 轉角 蓋 一座 城堡 我倆 好好 好 到 瘋 掉 像 找回 失散多年 雙胞 生命 再長 不過 煙火 落下 了 眼角 世界 再大 不過 你 我 凝視 的 微笑 在 所有 流逝 風景 與 人群 中 你 對 我 最好 一切 好好 是否 太好 沒有 人 知道 你 和 我 背著 空蕩 的 書包 逃避 名為 日常 的 監牢 忘記 了 要 長大 忘記 了 要 變老 忘記 了 時間 有腳 最 安靜 的 時刻 回憶 總是 最 喧囂 最 喧囂 的 狂歡 孤單 包圍 著 孤島 還以 為 馴服 想 能 陪伴 我 像 一隻 家貓 它 就 窩 在 沙發 一角 卻 不肯 睡著 你 和 我 曾經 有 滿滿的 羽毛 跳 著名 為 青鳥 的 舞蹈 不 知道 未來 不 知道 煩惱 不知道 那些 日子 會 是 那麼 少 時間 的 電影 結果 才 知道 原來 大人 已經 沒有 童謠 最後 的 叮嚀 最後 的 擁抱 我倆 紅著 眼笑 我倆 都 要 把 自己 照顧 好 好 到 遺憾 無法 打擾 好好 的 生活 好好 的 變老 好好 假裝 我 已經 把 你 忘記 

相似度： 0.996792

相似歌詞： 轉眼 走到 了 自傳 最終 章 已經 瀏覽 所有 命運 的 風光 混濁 的 瞳孔 風乾 的 皮囊 也 曾經 那般 花漾 最愛 的 相片 讓 你 挑 一張 千萬個 片刻 誰 在 你 身邊 那 一年 的 我 曾經 和 你 一樣 飛揚 惶惶不安 念念不忘 還是 得 放開 雙掌 手心 曾握 著 誰 的 體溫 漸涼 有沒有 人 在 某個 地方 等 我 重回 當初 的 樣子 雙頰 曾經 光滑 夜色 曾沁涼 世界 曾經 瘋狂 愛情 曾經 綻放 有沒有 人 依偎 我 身邊 聽 我 傾訴 餘生 的 漫長 在 你 的 眼中 我 似乎 健忘 因為 我 腦海 已有 最 難忘 最難 忘記 在 我 的 時代 還有 唱片 行 如同 博物館 裝滿 了 希望 披頭 與 槍花 愛情 和 憂傷 永遠 驕傲 高唱 成就 如 沙堡 生命 如 海浪 浪花 會 掏盡 所有 的 幻象 存款 與 樓房 掙扎 與 渴望 散場 回憶 如窗 冷淚 盈眶 風景 模糊 如 天堂 孤單 的 大床 誰 貼近 我 臉頰 有沒有 人 也 笑憶 過往 

## --END

### --補充：將 LSI 模型改為 LDA 模型實作

In [ ]:
# 創建 LDA model
lda = models.LdaModel(corpus_tfidf, id2word=dictionary, num_topics=20)
corpus_lda = lda[corpus_tfidf] # LDA潛在語義索引
lda.save('lyrics/lyrics_mayday.lda')
corpora.MmCorpus.serialize('lyrics/lda_corpus_mayday.mm', corpus_lda)
print("LDA topics:")
lda.print_topics(4)

In [ ]:
index = similarities.MatrixSimilarity(lda[corpus]) 
index.save("lyrics/lyrics_mayday.index") 

# 基於tfidf-> lda 的文本相似度分析
doc = "想 把 你 寫成 一首歌 想養 一隻 貓 想要 回到 每個 場景 撥慢 每 隻 錶 我們 在 小孩 和 大人 的 轉角 蓋 一座 城堡 我們 好好 好 到 瘋 掉 像 找回 失散多年 雙胞 生命 再長 不過 煙火 落下 了 眼角 世界 再大 不過 你 我 凝視 的 微笑 在 所有 流逝 風景 與 人群 中 你 對 我 最好 一切 好好 是否 太好 沒有 人 知道 你 和 我 背著 空空 的 書包 逃出 名為 日常 的 監牢 忘 了 要 長大 忘 了 要 變老 忘 了 時間 有腳 最 安靜 的 時刻 回憶 總是 最 喧囂 最 喧囂 的 狂歡 寂寞 包圍 著 孤島 還以 為 馴服 想念 能 陪伴 我 像 一隻 家貓 它 就 窩 在 沙發 一角 卻 不肯 睡著 你 和 我 曾 有 滿滿的 羽毛 跳 著名 為 青春 的 舞蹈 不 知道 未來 不 知道 煩惱 不知 那些 日子 會 是 那麼 少 時間 的 電影 結局 才 知道 原來 大人 已 沒有 童謠 最後 的 叮嚀 最後 的 擁抱 我們 紅著 眼笑 我們 都 要 把 自己 照顧 好 好 到 遺憾 無法 打擾 好好 的 生活 好好 的 變老 好好 假裝 我 已經 把 你 忘掉 "
vec_bow = dictionary.doc2bow(doc.split()) # 把doc語料庫轉為一個一個詞包
vec_lda = lda[vec_bow] # 用前面建好的 lsi 去計算這一篇歌詞

sims = index[vec_lda] # 將已經算完tfidf的字詞轉為lsi #基於lsi的文本相似度分析
sims = sorted(enumerate(sims), key=lambda item: -item[1])
print(sims[:5])

In [ ]:
lyrics = [];
fp = open("lyrics/lyrics_word_net_mayday.dataset") # 斷詞後的歌詞
#fp = open("lyrics/lyrics.dataset") ＃ 看完整的歌詞
for i, line in enumerate(fp):
    lyrics.append(line)
fp.close()

for lyric in sims[:5]:
    print("\n相似歌詞：",  lyrics[lyric[0]])
    print("相似度：",  lyric[1])